# small course on the inverse theory

jan 21-23 2026 @ Lyon with IsaVeroSteph 

inspired by the memo prepared by Isa Yael, you can look at the symbolic and concrete way of looking at the normal equations

and how the gradient direction, hessian and fréchet derivatives are linked in the linearised system

we can even put some arbitrary non-linear analytical expressions for u(m) 

which can be useful for my class next year :-)


In [2]:
using Pkg

cd(@__DIR__)
Pkg.activate("../")

  Activating project at `~/Documents/Github/flexOPT`


In [3]:
using Symbolics
include("../src/commonBatchs.jl")
using .commonBatchs

problem setting

In [4]:
# dimensions
modelDimension = 2
dataDimension  = 3

# models (nₑ distribution in the Earth)
@variables m[1:modelDimension] # a model
@variables mTrue[1:modelDimension] # the ground truth
@variables m⁰[1:modelDimension] # our initial guess (PREM or 3D initial model)
@variables δm[1:modelDimension] # an arbitrary model perturbation from m₀ : m = m₀+δm
@variables Δm[1:modelDimension]
δm = collect(δm)
Δm =[mTrue[i] - m⁰[i] for i ∈ 1:modelDimension] # model perturbation required to have the goround truth
#δm =[m[i] - m⁰[i] for i ∈ 1:modelDimension]
# data (predicted and observed), since we mimic 'symbolic partial derivations' by multiplications, u=G(m) stays only in our heart
# maybe WHATEVER!!!
@variables u(m...)[1:dataDimension]
@variables d[1:dataDimension] 
@variables Δd[1:dataDimension] # data - u(m⁰) but I will substitute at the end

δd = [d[i] - u[i] for i ∈ 1:dataDimension] # data residual for synthetic data predicted for an arbitrary model m = m₀+δm

# partials (in a wild symbolic way), but all the ∇s are linear so let's be super super simple (I don't really use the functionalities of )

#@variables ∂m[1:modelDimension]
#@variables ∂δm[1:modelDimension]
#@variables ∂m⁰[1:modelDimension]
∂m = [Differential(m[i]) for i ∈ 1:modelDimension]
∂m⁰ = [Differential(m⁰[i]) for i ∈ 1:modelDimension]
∂δm = [Differential(δm[i]) for i ∈ 1:modelDimension]

2-element Vector{Differential}:
 Differential(δm[1])
 Differential(δm[2])

In [ ]:
#cost function for F(m)
F = expand(1//2*sum(δd[i]^2 for i in 1:dataDimension))

Let's develop a Taylor expansion 

In [ ]:
# first derivative = gradient direction
∇F = [expand_derivatives(∂m[i](F)) for i ∈ 1:modelDimension] # sorry that ∂ can be found after the mathematical expressions but here it's treated as variables and it's ok to manipulate manually

In [ ]:
∇∇F = [expand_derivatives(∂m[j](∇F[i])) for i ∈ 1:modelDimension, j ∈ 1:modelDimension ] # second derivatives

In [ ]:

# if you don't do this, just comment it to see what's happening at the end
ignore_second_derivatives = Dict(∂m[i](∂m[j](u[k]))=>0 for i ∈ 1:modelDimension, j ∈ 1:modelDimension, k ∈ 1:dataDimension) ; 

In [ ]:
∇∇F = expand.(substitute.(∇∇F, Ref(ignore_second_derivatives)))

# substitution for $m^0$

In [ ]:
dict_for_m⁰ = Dict(m[i] => m⁰[i] for i ∈ 1:modelDimension)
dict_for_∂m⁰ = Dict(∂m[i] => ∂m⁰[i] for i ∈ 1:modelDimension)
dict_for_m⁰= merge(dict_for_m⁰,dict_for_∂m⁰)

In [ ]:
F⁰= substitute(F,dict_for_m⁰)

In [ ]:
∇F⁰= expand.(substitute.(∇F,Ref(dict_for_m⁰))) 

In [ ]:


∇∇F⁰= expand.(substitute.(∇∇F,Ref(dict_for_m⁰)))

In [ ]:
# we want to develop a Taylor expansion 
F_Taylor⁰ = F⁰ + expand(∇F⁰'*δm) + expand(1//2*δm'*∇∇F⁰*δm) # and δm^3 -> ignored

We are just looking for $\delta m$ which can make $F^0$ has its plateau
\begin{equation}
\nabla_{\delta m}  F = 0
\end{equation}

In [ ]:
∇F_Taylor⁰ = [expand_derivatives(expand(∂δm[i](F_Taylor⁰))) for i ∈ 1:modelDimension]

In [ ]:
dict_introduce_Δd = Dict(d[i]=> u[i]+Δd[i] for i ∈ 1:dataDimension)

In [ ]:
∇F_Taylor⁰ = expand.(Symbolics.scalarize(substitute.(∇F_Taylor⁰,Ref(dict_introduce_Δd ))))
∇F_Taylor⁰ = substitute.(∇F_Taylor⁰,Ref(dict_for_m⁰))

In [ ]:
# relation between the gradient direction and A⁰Δd

∇F⁰ = expand.(substitute.(∇F,Ref(dict_introduce_Δd)))
∇F⁰ = substitute.(∇F⁰,Ref(dict_for_m⁰))


In [ ]:

# check that the gradient direction has - Fréchet derivative times Δd
A⁰=Symbolics.jacobian(∇F⁰,Δd)'

In [ ]:
hessianContribution = ∇F_Taylor⁰ - ∇F⁰

In [ ]:
H = Symbolics.jacobian(hessianContribution,δm)

In [ ]:
H-A⁰'*A⁰